In [28]:
from theano import function, config, shared, tensor
import numpy as np
import theano
import time
import sys

import load as read_data
import custom_config as custom_config
import activations as activations
import utils as utils

In [29]:
## Load Data ##
datasets_dir = custom_config.datasets_dir

trX, teX, trY, teY = read_data.mnist(onehot=False)

trX=trX[0:1000,]
trY=trY[0:1000]
teX=teX[0:100,]
teY=teY[0:100]

print "Training data: ", trX.shape, trY.shape
print "Test data: ", teX.shape, teY.shape

/Users/achanish/Dropbox/My_Deep_Learning_Library/seq2seq/svdseq2seq/../data/mnist/
Training data:  (1000, 784) (1000,)
Test data:  (100, 784) (100,)


In [20]:
dim = 3
H = np.eye(dim)  
print(H)
D = np.ones(dim,)
print(D)
for n in range(1,dim):
    print n
#n=2
x = custom_config.rng.normal(size=(dim-n+1,)) #.normal(size=(dim-n+1,))
D[n-1] = np.sign(x[0])
#print x
print np.sign(x[1])

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[ 1.  1.  1.]
1
2
1.0


In [21]:
def rvs(random_state,dim):
     H = np.eye(dim)
     D = np.ones((dim,))
     for n in range(1, dim):
         x = random_state.normal(size=(dim-n+1,))
         D[n-1] = np.sign(x[0])
         x[0] -= D[n-1]*np.sqrt((x*x).sum())
         # Householder transformation
         Hx = (np.eye(dim-n+1) - 2.*np.outer(x, x)/(x*x).sum())
         mat = np.eye(dim)
         mat[n-1:, n-1:] = Hx
         H = np.dot(H, mat)
         # Fix the last sign such that the determinant is 1
     D[-1] = (-1)**(1-(dim % 2))*D.prod()
     # Equivalent to np.dot(np.diag(D), H) but faster, apparently
     H = (D*H.T).T
     return H

In [24]:
H = rvs(custom_config.rng , 3)
print(H)
print(np.dot(H, np.transpose(H)))

[[-0.63063109 -0.61462969 -0.47385099]
 [-0.30221721  0.75685889 -0.57950787]
 [ 0.71482108 -0.22224976 -0.66305042]]
[[  1.00000000e+00   1.66533454e-16   1.66533454e-16]
 [  1.66533454e-16   1.00000000e+00   1.11022302e-16]
 [  1.66533454e-16   1.11022302e-16   1.00000000e+00]]
